In [1]:
import numpy as np

PATTERN_TO_DEBUG = 0

def toDec(hexstr,BIT):
    msb4bits = hexstr[0]
    n = int(msb4bits, 16)
    if n >= 8:
        p = -1*pow(2,BIT-1)
        addend = int(str(n-8) + hexstr[1:], 16)
        return str( p + addend)
    else:
        return str(int(hexstr, 16))

def int2hex(num,bit_width):
    if num<0:
        num = (1 << bit_width) + num
    hex_string = hex(num)[2:]
    return '{:0>{width}}'.format(hex_string,width = (bit_width+3)//4)

NEGATE = {'1': '0', '0': '1'}
def negate(value):
    return ''.join(NEGATE[x] for x in value)

def twocomplement(n, size_in_bits):
    # print(n)
    number = int(n)
    if number < 0:
        return negate(bin(abs(number) - 1)[2:]).rjust(size_in_bits, '1')
    else:
        return bin(number)[2:].rjust(size_in_bits, '0')

NEGATE = {'1': '0', '0': '1'}
def negate(value):
    return ''.join(NEGATE[x] for x in value)

def reverseRow(data, index):

    cols = len(data[index])
    for i in range(cols // 2):
        temp = data[index][i]
        data[index][i] = data[index][cols - i - 1]
        data[index][cols - i - 1] = temp

    return data

# Rotate Matrix by 180 degrees
def rotateMatrix(data):

    rows = len(data)
    cols = len(data[0])

    if (rows % 2):
        # If N is odd reverse the middle
        # row in the matrix
        data = reverseRow(data, len(data) // 2)

        # Swap the value of matrix [i][j] with
        # [rows - i - 1][cols - j - 1] for half
        # the rows size.
        for i in range(rows // 2):
            for j in range(cols):
                temp = data[i][j]
                data[i][j] = data[rows - i - 1][cols - j - 1]
                data[rows - i - 1][cols - j - 1] = temp
        return data

def twocomplement(n, size_in_bits):
    # print(n)
    number = int(n)
    if number < 0:
        return negate(bin(abs(number) - 1)[2:]).rjust(size_in_bits, '1')
    else:
        return bin(number)[2:].rjust(size_in_bits, '0')


In [2]:
with open('input.txt', 'r') as file:
  file_in = file.readlines()

NUM = int(file_in[0])

# print(file_in)
for i in range(NUM):
  img_16 = []
  kernal_16 = []
  matrix_idx = []
  mode = []

  # MATRIX SIZE
  matrix_in_idx = int(file_in[1 + 65 * i + 0])

  if matrix_in_idx == 0:
     matrix_size = 8
  elif matrix_in_idx == 1:
     matrix_size = 16
  else:
     matrix_size = 32

  for j in range(16):
    img_in = np.array([int(toDec(val,8)) for val in file_in[1 + 65 * i + (1+j)].split()])
    img_16.append(np.reshape(img_in,(matrix_size,matrix_size)))

  for k in range(16):
    kernal_in = np.array([int(toDec(val,8)) for val in file_in[1 + 65 * i + (17+k)].split()])
    kernal_16.append(np.reshape(kernal_in,(5,5)))


  for idx in range(16):
    mode.append(int(file_in[1 + 65 * i + (33+idx*2)])) #0 2 4
    matrix_idx.append(np.array([int(val) for val in file_in[1 + 65 * i + (34+idx*2)].split()]))# 1 3 5

  if i == PATTERN_TO_DEBUG:
    break

In [3]:
mode[0]

0

In [4]:
matrix_idx[6]

array([13, 10])

In [5]:
img_16[0]

array([[ 3,  0, -3, -3, -3,  3,  1, -3],
       [ 2,  2, -3, -3,  3, -1, -2, -3],
       [-3,  3, -3, -1,  2,  1,  0,  1],
       [ 0, -2, -3, -2,  2, -3,  3, -3],
       [ 1,  0, -2, -3,  2,  1,  0, -3],
       [ 1,  2, -3,  1,  1, -3, -1,  3],
       [ 1, -3,  2, -1,  0,  3, -1, -1],
       [-2,  2, -2,  0,  1,  2, -3,  1]])

In [6]:
kernal_16[0]

array([[ 2,  0,  2,  1, -2],
       [ 1, -3, -1,  1,  3],
       [ 2,  3,  1,  3,  0],
       [-1,  1,  1, -3,  2],
       [ 1,  2, -1,  3,  1]])

In [7]:
matrix_idx[0]

array([0, 0])

In [8]:
matrix_size

8

## Convolution

In [9]:
img = img_16[13]
kernal = kernal_16[9]

convoluted_results = np.zeros((matrix_size-4,matrix_size-4))
for i in range(matrix_size-4):
    for j in range(matrix_size-4):
        conv = 0
        for k in range(5):
            for l in range(5):
                conv += kernal[k,l] * img[i+k,j+l]

        convoluted_results[i,j] = conv

In [10]:
convoluted_results

array([[ 18.,  -1.,  -7.,  17.],
       [-14.,   4.,  -6.,  18.],
       [ 17.,  -3.,   5., -23.],
       [-38.,  14.,  -7.,  12.]])

## Maxpooling

In [11]:
convoluted_img_width = len(convoluted_results)

In [12]:
max_pooling_results = np.zeros((convoluted_img_width//2,convoluted_img_width//2))

for i in range(0,convoluted_img_width,2):
    for j in range(0,convoluted_img_width,2):
        max_pooling_results[i//2,j//2] = max(convoluted_results[i,j],convoluted_results[i+1,j],\
                                                convoluted_results[i+1,j+1],convoluted_results[i,j+1])

In [13]:
max_pooling_results

array([[18., 18.],
       [17., 12.]])

### Merged Convolution and max pooling

In [14]:
img_16[0]

array([[ 3,  0, -3, -3, -3,  3,  1, -3],
       [ 2,  2, -3, -3,  3, -1, -2, -3],
       [-3,  3, -3, -1,  2,  1,  0,  1],
       [ 0, -2, -3, -2,  2, -3,  3, -3],
       [ 1,  0, -2, -3,  2,  1,  0, -3],
       [ 1,  2, -3,  1,  1, -3, -1,  3],
       [ 1, -3,  2, -1,  0,  3, -1, -1],
       [-2,  2, -2,  0,  1,  2, -3,  1]])

In [15]:
kernal_16[0]

array([[ 2,  0,  2,  1, -2],
       [ 1, -3, -1,  1,  3],
       [ 2,  3,  1,  3,  0],
       [-1,  1,  1, -3,  2],
       [ 1,  2, -1,  3,  1]])

In [16]:
img = img_16[0]
kernal = kernal_16[0]

In [17]:
max_pooling_results = np.zeros((convoluted_img_width//2,convoluted_img_width//2))
for img_yptr in range(0,matrix_size-4,2):
    for img_xptr in range(0,matrix_size-4,2):
        # print(img_yptr)

        temp_max = -999999
        # 4 times convolution to quickly get values needed for max pooling
        for mp_window_y in range(2):
            for mp_window_x in range(2):
                conv = 0
                idx_x = img_xptr+mp_window_x
                idx_y = img_yptr+mp_window_y
                for k_yptr in range(5):

                    temp = 0
                    # Uses High level modeling for fast MAC, img_x_cord[x] and kernal_x_cord[x]
                    for k_xptr in range(5): # Unrolling the x_ptr
                        conv += kernal[k_yptr,k_xptr] * img[idx_y+k_yptr,idx_x+k_xptr]
                        temp += kernal[k_yptr,k_xptr] * img[idx_y+k_yptr,idx_x+k_xptr]

                    # print(temp)
                    print(conv)

                if (mp_window_y == 0 and mp_window_x == 0) or conv > temp_max:
                    temp_max = conv

        max_pooling_results[img_yptr//2,img_xptr//2] = temp_max

3
5
-3
5
-4
-15
14
2
-15
6
-11
-4
-15
10
12
3
18
-9
-9
-5
-11
-4
-4
18
-7
7
-22
5
-14
-3
3
-1
-19
-2
-12
-4
-5
8
0
6
-17
13
-9
-3
-10
4
2
-3
-13
5
-12
6
8
1
5
0
14
-1
10
3
-1
7
-8
12
8
-2
-11
1
6
-8
-11
6
-11
-14
0
-1
-19
-1
5
-8


In [18]:
max_pooling_results

array([[ 6.,  5.],
       [26., 18.]])

In [19]:
max_pooling_results = np.zeros((convoluted_img_width//2,convoluted_img_width//2))
for img_yptr in range(0,matrix_size-4,2):
    for img_xptr in range(0,matrix_size-4,2):
        temp_max = -999999
        # 4 times convolution to quickly get values needed for max pooling
        for mp_window_y in range(2):
            for mp_window_x in range(2):
                conv = 0
                idx_y = img_yptr+mp_window_y
                idx_x = img_xptr+mp_window_x
                for k_yptr in range(5):
                    conv += kernal[k_yptr,0] * img[idx_y+k_yptr,idx_x+0]
                    conv += kernal[k_yptr,1] * img[idx_y+k_yptr,idx_x+1]
                    conv += kernal[k_yptr,2] * img[idx_y+k_yptr,idx_x+2]
                    conv += kernal[k_yptr,3] * img[idx_y+k_yptr,idx_x+3]
                    conv += kernal[k_yptr,4] * img[idx_y+k_yptr,idx_x+4]

                if (mp_window_y == 0 and mp_window_x == 0) or conv > temp_max:
                    temp_max = conv

        max_pooling_results[img_yptr//2,img_xptr//2] = temp_max

In [20]:
max_pooling_results

array([[ 6.,  5.],
       [26., 18.]])

### Transposed convolution

In [21]:
trans_convoluted_results = np.zeros((matrix_size+4,matrix_size+4))
for i in range(matrix_size):
    for j in range(matrix_size):
        for k in range(5):
            for l in range(5):
                conv = kernal[k,l] * img[i,j]
                trans_convoluted_results[i+k,j+l] += conv

In [22]:
trans_convoluted_results

array([[  6.,   0.,   0.,  -3., -18.,  -3.,  -1.,   3.,  11., -11.,  -5.,
          6.],
       [  7.,  -5.,  -8.,   9.,  16.,  -3., -12., -20.,  -9.,   7.,   1.,
         -3.],
       [  2.,  11., -26.,   1.,  12., -28.,  -6.,  -3.,  17.,  -2., -17.,
        -11.],
       [ -2.,  21.,  -7., -22.,   1., -12., -12.,  -1., -28.,  -4.,  -6.,
          3.],
       [ -3.,   1.,   2., -17.,   0., -19.,  11., -31.,  15.,   0.,   3.,
        -12.],
       [  8.,  -3., -17.,   1., -12., -10., -33., -14.,  -8.,  22., -21.,
        -16.],
       [  2.,  -5.,   0., -29.,   6.,  15., -22.,  18.,   7., -43.,  10.,
          6.],
       [ -2.,   4.,  -1., -11.,   3., -20., -18.,   8., -18.,  12.,  15.,
        -14.],
       [  0.,   6.,  -7.,  -3., -19.,  34., -33.,  -2.,  22.,   2., -31.,
          6.],
       [ -4.,   6.,  -4., -17.,  31., -24.,   4.,   7.,  -6.,  -9.,  12.,
          1.],
       [  3.,  -5.,  -3.,  15., -25.,  16.,   6., -12.,   2.,  15., -13.,
          1.],
       [ -2.,  -2.,  

## Transpoed convolution with zero-padding

In [23]:
img = img_16[0]

In [24]:
img

array([[ 3,  0, -3, -3, -3,  3,  1, -3],
       [ 2,  2, -3, -3,  3, -1, -2, -3],
       [-3,  3, -3, -1,  2,  1,  0,  1],
       [ 0, -2, -3, -2,  2, -3,  3, -3],
       [ 1,  0, -2, -3,  2,  1,  0, -3],
       [ 1,  2, -3,  1,  1, -3, -1,  3],
       [ 1, -3,  2, -1,  0,  3, -1, -1],
       [-2,  2, -2,  0,  1,  2, -3,  1]])

In [25]:
kernal = kernal_16[0]

In [26]:
kernal

array([[ 2,  0,  2,  1, -2],
       [ 1, -3, -1,  1,  3],
       [ 2,  3,  1,  3,  0],
       [-1,  1,  1, -3,  2],
       [ 1,  2, -1,  3,  1]])

In [27]:
new_trans_convoluted_results = np.zeros((matrix_size+4,matrix_size+4))
# Note the matrix is flipped
lower_bound0 = matrix_size + 4
lower_bound1 = matrix_size + 5
lower_bound2 = matrix_size + 6
lower_bound3 = matrix_size + 7

for img_ytr in range(matrix_size+4):
    for img_xptr in range(matrix_size+4):
        conv = 0
        for k_yptr in range(5):
            # Uses High level always@* , need idx_x[x]
            for k_xptr in range(5):
                idx_y = img_ytr + k_yptr
                idx_x = img_xptr + k_xptr
                if idx_y == 0 or idx_y == 1 or idx_y == 2 or idx_y == 3 or idx_x == 0 or idx_x == 1 \
                    or idx_x == 2 or idx_x == 3:
                        conv += 0
                elif idx_y == lower_bound0 or idx_y == lower_bound1 or idx_y == lower_bound2 or idx_y == lower_bound3 \
                    or idx_x == lower_bound0 or idx_x == lower_bound1 or idx_x == lower_bound2 or idx_x == lower_bound3 :
                        conv += 0
                else:
                   # Address translation
                   conv += kernal[4-k_yptr,4-k_xptr] * img[idx_y-4,idx_x-4]

        new_trans_convoluted_results[img_ytr,img_xptr] = conv


In [28]:
kernal

array([[ 2,  0,  2,  1, -2],
       [ 1, -3, -1,  1,  3],
       [ 2,  3,  1,  3,  0],
       [-1,  1,  1, -3,  2],
       [ 1,  2, -1,  3,  1]])

In [29]:
rotateMatrix(kernal)

array([[ 1,  3, -1,  2,  1],
       [ 2, -3,  1,  1, -1],
       [ 0,  3,  1,  3,  2],
       [ 3,  1, -1, -3,  1],
       [-2,  1,  2,  0,  2]])

In [30]:
new_trans_convoluted_results

array([[  6.,   0.,   0.,  -3., -18.,  -3.,  -1.,   3.,  11., -11.,  -5.,
          6.],
       [  7.,  -5.,  -8.,   9.,  16.,  -3., -12., -20.,  -9.,   7.,   1.,
         -3.],
       [  2.,  11., -26.,   1.,  12., -28.,  -6.,  -3.,  17.,  -2., -17.,
        -11.],
       [ -2.,  21.,  -7., -22.,   1., -12., -12.,  -1., -28.,  -4.,  -6.,
          3.],
       [ -3.,   1.,   2., -17.,   0., -19.,  11., -31.,  15.,   0.,   3.,
        -12.],
       [  8.,  -3., -17.,   1., -12., -10., -33., -14.,  -8.,  22., -21.,
        -16.],
       [  2.,  -5.,   0., -29.,   6.,  15., -22.,  18.,   7., -43.,  10.,
          6.],
       [ -2.,   4.,  -1., -11.,   3., -20., -18.,   8., -18.,  12.,  15.,
        -14.],
       [  0.,   6.,  -7.,  -3., -19.,  34., -33.,  -2.,  22.,   2., -31.,
          6.],
       [ -4.,   6.,  -4., -17.,  31., -24.,   4.,   7.,  -6.,  -9.,  12.,
          1.],
       [  3.,  -5.,  -3.,  15., -25.,  16.,   6., -12.,   2.,  15., -13.,
          1.],
       [ -2.,  -2.,  

In [31]:
np.array_equal(new_trans_convoluted_results,trans_convoluted_results)

True

In [32]:
new_trans_convoluted_results = np.zeros((matrix_size+4,matrix_size+4))
lower_bound0 = matrix_size + 4
lower_bound1 = matrix_size + 5
lower_bound2 = matrix_size + 6
lower_bound3 = matrix_size + 7

for img_ytr in range(matrix_size+4):
    for img_xptr in range(matrix_size+4):
        conv = 0
        if(img_yptr ==2 and img_xptr == 0): print("DEBUG")
        for k_yptr in range(5):
            # for k_xptr in range(5):
                idx_y = img_ytr + k_yptr
                idx_x_0 = img_xptr + 0
                idx_x_1 = img_xptr + 1
                idx_x_2 = img_xptr + 2
                idx_x_3 = img_xptr + 3
                idx_x_4 = img_xptr + 4
                # 0
                if idx_y == 0 or idx_y == 1 or idx_y == 2 or \
                    idx_y == 3 or idx_x_0 == 0 or idx_x_0 == 1 \
                    or idx_x_0 == 2 or idx_x_0 == 3:
                        conv += 0
                elif idx_y == lower_bound0 or idx_y == lower_bound1 or idx_y == lower_bound2 or idx_y == lower_bound3 \
                    or idx_x_0 == lower_bound0 or idx_x_0 == lower_bound1 or idx_x_0 == lower_bound2 or idx_x_0 == lower_bound3 :
                        conv += 0
                else:
                   conv += kernal[4-k_yptr,4] * img[idx_y-4,idx_x_0-4]
                # 1
                if idx_y == 0 or idx_y == 1 or idx_y == 2 or \
                    idx_y == 3 or idx_x_1 == 0 or idx_x_1 == 1 \
                    or idx_x_1 == 2 or idx_x_1 == 3:
                        conv += 0
                elif idx_y == lower_bound0 or idx_y == lower_bound1 or idx_y == lower_bound2 or idx_y == lower_bound3 \
                    or idx_x_1 == lower_bound0 or idx_x_1 == lower_bound1 or idx_x_1 == lower_bound2 or idx_x_1 == lower_bound3 :
                        conv += 0
                else:
                   conv += kernal[4-k_yptr,3] * img[idx_y-4,idx_x_1-4]
                # 2
                if idx_y == 0 or idx_y == 1 or idx_y == 2 or \
                    idx_y == 3 or idx_x_2 == 0 or idx_x_2 == 1 \
                    or idx_x_2 == 2 or idx_x_2 == 3:
                        conv += 0
                elif idx_y == lower_bound0 or idx_y == lower_bound1 or idx_y == lower_bound2 or idx_y == lower_bound3 \
                    or idx_x_2 == lower_bound0 or idx_x_2 == lower_bound1 or idx_x_2 == lower_bound2 or idx_x_2 == lower_bound3 :
                        conv += 0
                else:
                   conv += kernal[4-k_yptr,2] * img[idx_y-4,idx_x_2-4]
                # 3
                if idx_y == 0 or idx_y == 1 or idx_y == 2 or \
                    idx_y == 3 or idx_x_3 == 0 or idx_x_3 == 1 \
                    or idx_x_3 == 2 or idx_x_3 == 3:
                        conv += 0
                elif idx_y == lower_bound0 or idx_y == lower_bound1 or idx_y == lower_bound2 or idx_y == lower_bound3 \
                    or idx_x_3 == lower_bound0 or idx_x_3 == lower_bound1 or idx_x_3 == lower_bound2 or idx_x_3 == lower_bound3 :
                        conv += 0
                else:
                   conv += kernal[4-k_yptr,1] * img[idx_y-4,idx_x_3-4]

                # 4
                if idx_y == 0 or idx_y == 1 or idx_y == 2 or \
                    idx_y == 3 or idx_x_4 == 0 or idx_x_4 == 1 \
                    or idx_x_4 == 2 or idx_x_4 == 3:
                        conv += 0
                elif idx_y == lower_bound0 or idx_y == lower_bound1 or idx_y == lower_bound2 or idx_y == lower_bound3 \
                    or idx_x_4 == lower_bound0 or idx_x_4 == lower_bound1 or idx_x_4 == lower_bound2 or idx_x_4 == lower_bound3 :
                        conv += 0
                else:
                   conv += kernal[4-k_yptr,0] * img[idx_y-4,idx_x_4-4]

        new_trans_convoluted_results[img_ytr,img_xptr] = conv


DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG


In [33]:
new_trans_convoluted_results

array([[  3.,   9.,  -6.,  -6.,  -6.,  -9.,   4., -12.,  -7.,   8.,  -5.,
         -3.],
       [  8.,  -1.,  -2.,  -4.,   0.,  16., -27., -11.,  10.,  -7., -12.,
          0.],
       [  1.,   1.,   2., -12.,  11., -35., -11.,  -1.,   5.,   2.,  -6.,
         -2.],
       [  3.,  22., -31., -24.,  -3.,   5., -14.,   8., -44.,  -2.,  -5.,
        -13.],
       [  1.,   1.,   6., -36.,   6.,  -8.,  -8., -27.,  10.,  18.,   0.,
         -7.],
       [-10.,   6.,   2., -11.,  -8.,  -2., -31.,   5.,   1., -20.,  -8.,
         -5.],
       [  9., -11., -26.,  19., -26.,  -5.,  18., -14.,  -2.,   1.,   4.,
        -11.],
       [  3.,  -5.,  24., -43.,   8.,  26., -30., -24.,  20., -24.,  21.,
         -1.],
       [ -3.,  21., -22.,  11., -11.,  -8., -11.,  23.,  15.,  -9., -11.,
         -6.],
       [  1., -17.,  16., -12.,   2.,  15.,   3., -23.,   2.,   3.,  -3.,
          7.],
       [ -8.,  11.,  -7.,   0.,   2.,   1.,  -1.,   0.,  -4.,  14.,  -8.,
         -1.],
       [  4.,  -6.,  

#### Convert results to LSB binary representation

In [34]:
twocomplement(57,20)

'00000000000000111001'

In [35]:
vf = np.vectorize(twocomplement)
bits_result = vf(trans_convoluted_results,20)

In [36]:
bits_result[0][0]

'00000000000000000110'

In [37]:
result = bits_result[0][0]

In [38]:
result = result[::-1]

In [39]:
result

'01100000000000000000'